In [1]:
import pandas as pd
from tableone import TableOne
import matplotlib.pyplot as plt
import numpy as np
import os 

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/conorcorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/google/protobuf/descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from google.protobuf.pyext import _message
/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [15]:
with open('table1.sql', 'r') as f:
    query = f.read()
query_job = client.query(query)
df=query_job.to_dataframe()

In [16]:
# df = pd.read_csv('demographics.csv')
# df.groupby('culture_growth').count()
df.sort_values('ANON_ID').tail(50)

,department_name,age,pat_enc_csn_id_coded,year,ANON_ID,GENDER,CANONICAL_RACE,CANONICAL_ETHNICITY,LANGUAGE,INSURANCE_PAYOR_NAME
7313,PEDIATRIC EMERGENCY DEPARTMENT,76,131212340303,2017,JCebe36e,Male,White,Non-Hispanic,English,Medicare
3298,PEDIATRIC EMERGENCY DEPARTMENT,54,131261254030,2018,JCebe573,Male,Other,Non-Hispanic,English,Other
2365,PEDIATRIC EMERGENCY DEPARTMENT,79,131259903812,2018,JCebe6e0,Female,White,Non-Hispanic,English,Medicare
8105,VCP EMERGENCY DEPARTMENT,87,131263826408,2019,JCebe72e,Male,White,Non-Hispanic,English,Medicare
6106,PEDIATRIC EMERGENCY DEPARTMENT,65,131243578394,2018,JCebec02,Male,White,Non-Hispanic,English,Other
5121,VCP EMERGENCY DEPARTMENT,88,131254093241,2018,JCebf06a,Female,White,Non-Hispanic,English,Medicare
2663,PEDIATRIC EMERGENCY DEPARTMENT,74,131201172972,2016,JCebf390,Female,Other,Non-Hispanic,English,Other
3054,PEDIATRIC EMERGENCY DEPARTMENT,64,131026878851,2013,JCebf91f,Female,Other,Non-Hispanic,English,Other
2796,PEDIATRIC EMERGENCY DEPARTMENT,85,131129951224,2015,JCebfc12,Female,Asian,Non-Hispanic,Non-English,Other
7954,VCP EMERGENCY DEPARTMENT,74,131272702990,2019,JCebfd91,Female,White,Non-Hispanic,English,Other


In [11]:
df.groupby('department_name').count()

,age,pat_enc_csn_id_coded,year,ANON_ID,GENDER,CANONICAL_RACE,CANONICAL_ETHNICITY,LANGUAGE,INSURANCE_PAYOR_NAME,culture_growth
department_name,,,,,,,,,,
ADULT EMERGENCY DEPARTMENT,94,94,94,94,94,94,94,94,94,94
PEDIATRIC EMERGENCY DEPARTMENT,7430,7430,7430,7430,7430,7430,7430,7430,7430,7430
VCP EMERGENCY DEPARTMENT,818,818,818,818,818,818,818,818,818,818


In [17]:
# Read in demographics table

#culture_growth=lambda x: ['Positive Cultures' if a == 1 else "Negative Cultures"
                                        #for a in x.culture_growth],
df = (df
      .assign(department_name=lambda x: ['Stanford ED' if 'VCP' not in a else "Valley Care ED"
                                         for a in x.department_name])
      .assign(split=lambda x: ['Train + Validation (2009-2018)' if year >= 2009 and year < 2019
                               else "Test (2019)" for year in x.year])
      .rename(columns={
        'department_name' : 'Emergency Department',
        "age" : "Age",
        "GENDER" : "Sex",
        "CANONICAL_RACE" : "Race",
        "CANONICAL_ETHNICITY" : "Ethnicity",
        "LANGUAGE" : "Language",
        "INSURANCE_PAYOR_NAME" : "Insurance Payer",
        "split" : "Dataset",
                      })
)
# Group by patient identifier to get average age 
df.head()

,Emergency Department,Age,pat_enc_csn_id_coded,year,ANON_ID,Sex,Race,Ethnicity,Language,Insurance Payer,Dataset
0,Stanford ED,57,131253022817,2018,JCea8c47,Female,White,Non-Hispanic,English,Other,Train + Validation (2009-2018)
1,Stanford ED,62,131254514837,2018,JCe260a9,Male,Black,Non-Hispanic,English,Other,Train + Validation (2009-2018)
2,Stanford ED,25,131254812668,2018,JCe2f36a,Male,Asian,Non-Hispanic,English,Other,Train + Validation (2009-2018)
3,Stanford ED,45,131256180674,2018,JCccde6e,Female,Other,Non-Hispanic,English,Medicare,Train + Validation (2009-2018)
4,Stanford ED,51,131258016748,2018,JCe28a75,Male,Other,Non-Hispanic,Non-English,Other,Train + Validation (2009-2018)


In [18]:
columns = ["Emergency Department", 'Age', 'Sex', 'Race', 'Ethnicity', "Language", "Insurance Payer"]
categorical = ['Emergency Department', 'Sex', 'Race', 'Ethnicity', "Language", "Insurance Payer"]
df = df.sort_values('Dataset', ascending=False)
df.head()
mytable = TableOne(df,
                   columns=columns,
                   categorical=categorical,
                   groupby='Dataset',
                   missing=False,
                   overall=False)


/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3584: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/conorcorbin/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [19]:
print(mytable.tabulate(tablefmt="github"))

|                             |                  | Test (2019)   | Train + Validation (2009-2018)   |
|-----------------------------|------------------|---------------|----------------------------------|
| n                           |                  | 1320          | 7022                             |
| Emergency Department, n (%) | Stanford ED      | 855 (64.8)    | 6669 (95.0)                      |
|                             | Valley Care ED   | 465 (35.2)    | 353 (5.0)                        |
| Age, mean (SD)              |                  | 70.4 (17.2)   | 67.5 (17.3)                      |
| Sex, n (%)                  | Female           | 793 (60.1)    | 4171 (59.4)                      |
|                             | Male             | 527 (39.9)    | 2851 (40.6)                      |
| Race, n (%)                 | Asian            | 201 (15.2)    | 937 (13.3)                       |
|                             | Black            | 69 (5.2)      | 464 (6.6)      

In [20]:

mytable.to_html('table1_by_dataset.html')

In [21]:
mytable.head()

Grouped by Dataset  \
                                                  Test (2019)   
n                                                        1320   
Emergency Department, n (%) Stanford ED            855 (64.8)   
                            Valley Care ED         465 (35.2)   
Age, mean (SD)                                    70.4 (17.2)   
Sex, n (%)                  Female                 793 (60.1)   

                                                                           
                                           Train + Validation (2009-2018)  
n                                                                    7022  
Emergency Department, n (%) Stanford ED                       6669 (95.0)  
                            Valley Care ED                      353 (5.0)  
Age, mean (SD)                                                67.5 (17.3)  
Sex, n (%)                  Female                            4171 (59.4)

In [21]:
mytable.to_latex()

'\\begin{tabular}{llll}\n\\toprule\n                       &       & \\multicolumn{2}{l}{Grouped by Microbial Culture Results} \\\\\n                       &       &                             Negative &     Positive \\\\\n\\midrule\nn & {} &                                21974 &         8342 \\\\\nEmergency Department, n (\\%) & Stanford ED &                         20173 (91.8) &  7524 (90.2) \\\\\n                       & Valley Care ED &                           1801 (8.2) &    818 (9.8) \\\\\nAge, mean (SD) &       &                          63.1 (18.4) &  67.9 (17.3) \\\\\nSex, n (\\%) & Female &                         10843 (49.3) &  4964 (59.5) \\\\\n                       & Male &                         11130 (50.7) &  3378 (40.5) \\\\\n                       & Unknown &                              1 (0.0) &              \\\\\nRace, n (\\%) & Asian &                          3218 (14.6) &  1138 (13.6) \\\\\n                       & Black &                           1555 

In [22]:
mytable.to_csv('table1_by_dataset.csv')

In [22]:
df_table = pd.read_csv('table_')

AttributeError: 'TableOne' object has no attribute 'df'